## Project 1

##### Team member: Zhihong Zhang.

The dataset of this project is based on the hflights package provided in the R programming language Package. 

#### The setting up of the project

In [2]:
#starting point:import the library and data set
import numpy as np
import pandas as pd

hflight ='https://raw.githubusercontent.com/jtopor/DAV-5400/master/Project1/hflights.csv'
flights = pd.read_csv(hflight,index_col=0)
# data set check - make sure data was read in as expected
flights.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted
Year,,,,,,,,,,,,,,,,,,,,
2011,1,1,6,1400.0,1500.0,AA,428,N576AA,60.0,40.0,-10.0,0.0,IAH,DFW,224,7.0,13.0,0,NaN,0
2011,1,2,7,1401.0,1501.0,AA,428,N557AA,60.0,45.0,-9.0,1.0,IAH,DFW,224,6.0,9.0,0,NaN,0
2011,1,3,1,1352.0,1502.0,AA,428,N541AA,70.0,48.0,-8.0,-8.0,IAH,DFW,224,5.0,17.0,0,NaN,0
2011,1,4,2,1403.0,1513.0,AA,428,N403AA,70.0,39.0,3.0,3.0,IAH,DFW,224,9.0,22.0,0,NaN,0
2011,1,5,3,1405.0,1507.0,AA,428,N492AA,62.0,44.0,-3.0,5.0,IAH,DFW,224,9.0,9.0,0,NaN,0


As the data base mentioned, the data set contains all ﬂights departing from Houston airports IAH (George Bush Intercontinental) and HOU (Houston Hobby) in 2011 .After checking the data, I decided to compare the performance of each airport.Since there are only two airports are involved, I may also compare with some airline performance within the airport. And Ideally I hope I can find the reasons behind the scenerio. 

According to this goal of comparision on performance, three research questions are created and I decide to test performance based on the cancelliation by airport or airline,time spending on the ground and the total delay of the flight. 

### Question 1 

The first question I want to discover is that which airport has more cancellation rate and how it is impacted.
By the experience of the flight, the canceliation is generally one of the worst conditions that affects the flight experience.

To explore the data, I firstly need to make sure how many cancellation are there in the data set. Due to the possibility that data set is not evenly distributed into two airports (which means that one airport will have more flight) To make comparision, the rate is much more preferred instead of the total number of cancellation. To get start, I filtered the data to get flight with cancellation.

In [3]:
#create a copy for question 1 to make sure not messing up data
flight1 = flights.copy()

#creates subset of the flights data based on step 1
Cancellation = flight1[ (flight1['Cancelled']==1) ]              
 #prints a sample of the data
Cancellation.head()
#check how many cancel of the whole data
print(Cancellation.count())


Month                224
DayofMonth           224
DayOfWeek            224
DepTime                9
ArrTime                0
UniqueCarrier        224
FlightNum            224
TailNum              116
ActualElapsedTime      0
AirTime                0
ArrDelay               0
DepDelay               9
Origin               224
Dest                 224
Distance             224
TaxiIn                 0
TaxiOut                4
Cancelled            224
CancellationCode     224
Diverted             224
dtype: int64


There are 224 cancellation among these data set. And it is not too bad if considering less than 1% of the whole data set. Then I want to know which airport contributes more cancellation.To make a fair comparision, I also need to check the total numbers of flights by each airport. Then I can compute the cancellation rate.

In [4]:
#group data
flight_airport = flight1.groupby(['Origin'])
#then get number of total flights by each airport
flight_per_airport = flight_airport['Origin'].count()
print(flight_per_airport )

Origin
HOU     4319
IAH    15681
Name: Origin, dtype: int64


In [5]:
Cancellation_airport=  Cancellation.loc[:,['Origin','CancellationCode']]
#filter to be IAH airport only
Cancellation_airport_IAH = Cancellation_airport[Cancellation_airport['Origin']=='IAH'] 
#filter to be HOU airport only
Cancellation_airport_HOU =Cancellation_airport[Cancellation_airport['Origin']=='HOU']
#print descriptive statistics for two airport canceliation
print(Cancellation_airport_IAH.describe())
print(Cancellation_airport_HOU.describe())
#compute Cancellation rate by airport
Cancellationrate_IAH= 156/15681
Cancellationrate_HOU=  68/4319
print(Cancellationrate_IAH)
print(Cancellationrate_HOU)

       Origin CancellationCode
count     156              156
unique      1                3
top       IAH                B
freq      156              122
       Origin CancellationCode
count      68               68
unique      1                3
top       HOU                B
freq       68               40
0.009948345131050315
0.015744385274369065


Among these 224 cancellations, there are 156 in IAH and 68 in HOU. For both airport, the weather is the main reason of canceliation, and it contributes 122 out of 156 cancellation in IAH and 40 out of 68 in HOU. Regarding the fact that HOU has 4319 flights in total and IAH has 15681 flights in Total,  HOU has bigger cancellation rate (1.57%)although it has lower total number of cancellations. I decide also explore more on the reasons of the cancellation.

In [6]:
#print percentage of weather contributing to canceliation for both airports
IAH_weather_rate=122/156 
print(IAH_weather_rate)
HOU_weather_rate=40/68
print(HOU_weather_rate)

0.782051282051282
0.5882352941176471


Based on the computation, IAH is impacted more by weather condition, and 78.2% cancellation is due to the weather.Later we can explore further on the airline data.

In [7]:
#create subset of carrier and reasons of cancellation
Cancellation_carrier = Cancellation.loc[:,['UniqueCarrier','CancellationCode']]
#filter cancellation to be cause by airline
Cancellation_carrier_A = Cancellation_carrier[Cancellation_carrier['CancellationCode']=='A']

#get most frequent airline fault
print(Cancellation_carrier_A.describe())


       UniqueCarrier CancellationCode
count             57               57
unique            11                1
top               WN                A
freq              23               57


To explore more on these airline data, we can make bar plot on it

In [8]:
#print bar plot (for some reason, the plot was showed in my Spyder instead of Juypter Notebook, and I assume the code is right)
print(Cancellation_carrier_A['UniqueCarrier'].value_counts().plot.bar())

Axes(0.125,0.125;0.775x0.755)


Based on Bar chart above,top three airline on cancellation is WN,XE and EV. And WN is almost twice as the XE. 

##### Result:

Based on the data above, IAH airport has a lower percentage of cancellation as compared with HOU.At the same time, HOU are less impacted by weather on cancellation.Among these airlines, WN, which is northwest, turns out to be the major airline make cancellation of their own reason.

### Question 2

Along with the cancellation, another important factor impacts the flight experience is the time spent on the ground which includes taxi in and taxi out time. Based on the dataset,this on-ground time is also the subtraction of actual elaspe time and air time.In this case, I choose the subtraction of totalelapse and airtime instead of the sum of taxi in and out to be my ground time.

For the second research question,I want to explore their onground time for each airport, and by definition, The average lower on ground time provide a better performance.

In [9]:
#create a copy for question 2 to make sure not messing up data
flight2 = flights.copy()
#creates Onground item in the data
flight2['Onground'] = (flight2['ActualElapsedTime'] - flight2['AirTime']) 
#create the subset of Onground time and airport only
Onground_time=flight2.loc[:,['Origin','Onground',]]
Onground_time.describe()

,Onground
count,19738.000000
mean,21.210964
std,8.493510
min,6.000000
25%,15.000000
50%,20.000000
75%,25.000000
max,176.000000


Read from the total data about both airports, it can be found that the mean is a bit greater than median, since the values are very close, the data is quite normal, and it is a bit skew to the right. And then I can seperate the data by airport).

In [10]:
#filter to be IAH airport only
Onground_time_airport_IAH = Onground_time[Onground_time['Origin']=='IAH'] 
 #filter to be HOU airport only
Onground_time_airport_HOU =Onground_time[Onground_time['Origin']=='HOU']
#print descriptive statistics for two airport canceliation
print(Onground_time_airport_IAH.describe())
print(Onground_time_airport_HOU.describe())

           Onground
count  15492.000000
mean      23.081655
std        8.219999
min        8.000000
25%       18.000000
50%       21.000000
75%       27.000000
max      176.000000
          Onground
count  4246.000000
mean     14.385539
std       5.428622
min       6.000000
25%      11.000000
50%      13.000000
75%      16.000000
max      74.000000


Based on descrptive statistics of two airport seperately, we can tell that the mean of on ground time is greater than median for IAH , while the mean is smaller than median for HOU. On the other hand, the onground time for IAH is skewed to the right, but the onground time for HOU is skewed to the left. 

##### Result:

As the data above showed, the HOU airport has a relatively low Onground time, and it means that customer can spend less extra time on travelling which gives better performance.

### Question 3

For the third question, I tried to find the total delay of each flight by airport. And the flight with lower delay value provides a better performance.Firstly, I create the extra column of "total delay", which is the sum of the arrive delay and depart delay. Also, to avoid meaningless cancelled flight, I also filtered out the cancelled flight.

In [11]:
#create a copy for question 3 to make sure not messing up data
flight3 = flights.copy()
#add 
#creates total delay item in the data
flight3['TotalDelay'] = (flight3['ArrDelay'] + flight3['DepDelay']) 


In [12]:
#creates subset of total delay and filter out the canceled flight.
totaldelay = flight3[(flight3['TotalDelay']>0) & (flight3['Cancelled']==0)]              
#prints a sample of the data
totaldelay.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,AirTime,...,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,TotalDelay
Year,,,,,,,,,,,,,,,,,,,,,
2011,1,4,2,1403.0,1513.0,AA,428,N403AA,70.0,39.0,...,3.0,IAH,DFW,224,9.0,22.0,0,NaN,0,6.0
2011,1,5,3,1405.0,1507.0,AA,428,N492AA,62.0,44.0,...,5.0,IAH,DFW,224,9.0,9.0,0,NaN,0,2.0
2011,1,9,7,1443.0,1554.0,AA,428,N476AA,71.0,41.0,...,43.0,IAH,DFW,224,8.0,22.0,0,NaN,0,87.0
2011,1,10,1,1443.0,1553.0,AA,428,N504AA,70.0,45.0,...,43.0,IAH,DFW,224,6.0,19.0,0,NaN,0,86.0
2011,1,11,2,1429.0,1539.0,AA,428,N565AA,70.0,42.0,...,29.0,IAH,DFW,224,8.0,20.0,0,NaN,0,58.0


After getting the total delay data, I then created bar charts by each airport.

In [13]:
#get the median of departure delay grouped by airport
totaldelay_median = totaldelay.groupby(['Origin']).median()['TotalDelay'] 
 #plots a bar chart for median total delay (similar to previous graph,work in my Spyder)
print(totaldelay_median.plot.bar())

Axes(0.125,0.125;0.775x0.755)


In [14]:
#get the mean of departure delay grouped by airport
totaldelay_median = totaldelay.groupby(['Origin']).mean()['TotalDelay'] 
 #plots a bar chart for mean total delay
print(totaldelay_median.plot.bar())

Axes(0.125,0.125;0.775x0.755)


Through these two bar charts, it is easy to tell that for both airport, the mean is significantly larger than median, which means that the data is skewed to the right. And these difference showed that there are some extreme values to make the data not quite normal. So for decide the average, I decide to use the median instead of the regular mean. Also,based on this bar chart, we can tell that HOU airport has higher average total delay.To have a complete view of statistical data, then I plot the descriptive statistics data for both airports. 

In [15]:
#filter to get IAH airport 
totaldelay_airport_IAH = totaldelay[totaldelay['Origin']=='IAH'] 
#filter to get HOU airport
totaldelay_airport_HOU =totaldelay[totaldelay['Origin']=='HOU']
#print descriptive statistics for two airport totaldelay
print(totaldelay_airport_IAH['TotalDelay'].describe())
print(totaldelay_airport_HOU['TotalDelay'].describe())

count    7349.000000
mean       36.062457
std        60.088941
min         1.000000
25%         7.000000
50%        17.000000
75%        40.000000
max      1555.000000
Name: TotalDelay, dtype: float64
count    2500.000000
mean       43.180400
std        52.211354
min         1.000000
25%        10.000000
50%        26.000000
75%        55.000000
max       487.000000
Name: TotalDelay, dtype: float64


These maximum and minimum value verified the assumption of the data and the dataset does spread a lot. 

##### Result:

Based on these data above, IAH has a lower average total delay value,and it is about 30% lower than HOU. In this case, the IAH definitely has a better performance.

### Conclusion

By answering three questions and explore various type of data, in my opinion,these three research questions provide the part of view how the customer may expect on the flight,which is stable on schedule,and travel on time.For the further exploration,if the data set can provide the revenue, and it may be also useful for computing the price per distance.Overall, I would like to say that IAH has a better performance overall.
HOU showed better performance on onground time but get relatively low score on total delay or cancellation rate. In cancelation data, since HOU has lower rate of weather caused cancellation(58.8%) than IAH's (78%), HOU has potential to improve the system on it for its relatively better weather conditions.Also, HOU need to work harder on dealing with total delay.The delay at HOU is quite dramatical. 